- 로그에 누락이 있을 수 있음
- 분할 결제가 있을 수 있음
- 분할 결제의 경우, 결제 완료 시간에서 payment_sequential의 내림차순으로 20~40초 사이의 임의의 시간만큼 차감

In [1]:
import pandas as pd
import os
import numpy as np
np.random.seed(42)
from datetime import timedelta

In [2]:
from pathlib import Path

REDEFINED_DIR = "../../downloads/olist_redefined"
os.makedirs(REDEFINED_DIR, exist_ok=True)

In [3]:
payments_origin = pd.read_csv("../../downloads/olist/olist_order_payments_dataset.csv")
orders_origin = pd.read_csv("../../downloads/olist/olist_orders_dataset.csv")
order_status = pd.read_csv("../../downloads/olist_redefined/order_status.tsv", sep='\t')
order_status_purchased = order_status[order_status['status'] == 'purchase']
customer_origin = pd.read_csv("../../downloads/olist/olist_customers_dataset.csv")

In [4]:
tmp = payments_origin.merge(orders_origin[['order_id', 'customer_id']], on='order_id', how='left')
tmp = tmp.merge(customer_origin[['customer_id', 'customer_unique_id']], on='customer_id', how='left')
tmp.drop(columns=['customer_id'], inplace=True)
tmp = tmp.rename(columns={'customer_unique_id': 'customer_id'})

payment_log = tmp.merge(order_status_purchased[['order_id', 'timestamp']], on='order_id', how='left')

print(len(order_status_purchased))
print(payment_log['order_id'].nunique())
print("ERROR 존재: 구매 기록은 있는데, 결제 기록이 없음")

99441
99440
ERROR 존재: 구매 기록은 있는데, 결제 기록이 없음


In [5]:
# Mock Payment Log 생성 및 처리
def create_mock_payment_log(payment_log):
    """
    결제 로그의 mock 데이터를 생성하여 중복 timestamp를 처리합니다.
    
    Args:
        payment_log: 원본 결제 로그 DataFrame
        
    Returns:
        DataFrame: 처리된 mock 결제 로그
    """
    # 1. 원본 데이터 복사 및 기본 전처리
    mock_log = payment_log.copy(deep=True)
    mock_log['timestamp'] = pd.to_datetime(mock_log['timestamp'], errors='coerce')
    
    # 2. 정렬 (timestamp, order_id, payment_sequential 순)
    mock_log = mock_log.sort_values(['timestamp', 'order_id', 'payment_sequential'])
    
    # 3. 중복 timestamp 식별
    mock_log['is_duplicate'] = mock_log.duplicated(subset=['timestamp'], keep=False)
    
    # 4. 중복 timestamp에 랜덤 시간 추가
    mock_log = add_random_time_to_duplicates(mock_log)
    
    return mock_log


def add_random_time_to_duplicates(df):
    """
    중복된 timestamp를 가진 결제에 대해 완료 시각에서 역순으로 랜덤 시간을 차감합니다.
    payment_sequential이 1이 아닌 경우, 가장 마지막 결제 시각에서 20~40초씩 순차 차감합니다.
    
    Args:
        df: 처리할 DataFrame
        
    Returns:
        DataFrame: 조정된 timestamp가 적용된 DataFrame
    """
    # 중복 timestamp 그룹별로 처리
    for timestamp_group in df[df['is_duplicate']]['timestamp'].unique():
        if pd.isna(timestamp_group):
            continue
            
        # 해당 timestamp를 가진 모든 행을 payment_sequential 순으로 정렬
        group_mask = (df['timestamp'] == timestamp_group)
        group_data = df[group_mask].copy()
        group_data = group_data.sort_values('payment_sequential')
        
        # 해당 그룹의 모든 payment_sequential 처리 (1개만 있으면 그대로 유지)
        if len(group_data) > 1:
            # 가장 큰 payment_sequential이 원본 완료 시간 (그대로 유지)
            max_payment_seq = group_data['payment_sequential'].max()
            completion_time = timestamp_group
            
            # payment_sequential을 큰 수부터 작은 수 순으로 정렬
            sorted_payments_origin = group_data.sort_values('payment_sequential', ascending=False)
            
            # 가장 큰 번호를 제외하고 순차적으로 시간 차감
            cumulative_seconds = 0
            for idx, row in sorted_payments_origin.iterrows():
                if row['payment_sequential'] == max_payment_seq:
                    # 가장 큰 payment_sequential은 원본 시간 유지
                    continue
                else:
                    # 20~40초 랜덤 시간 생성 후 누적
                    random_seconds = np.random.uniform(20, 40)
                    cumulative_seconds += random_seconds
                    
                    # 완료 시각에서 누적 시간만큼 차감
                    adjusted_time = completion_time - timedelta(seconds=cumulative_seconds)
                    adjusted_time = adjusted_time.replace(microsecond=0)  # 밀리초 절삭
                    df.loc[idx, 'timestamp'] = adjusted_time
    
    return df.drop(columns=['is_duplicate'])

In [6]:
mock_payment_log = create_mock_payment_log(payment_log)

In [7]:
payment_log[payment_log['order_id']=='63638a6806d67773f3adba8534553fff']

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,timestamp
7876,63638a6806d67773f3adba8534553fff,5,voucher,1,34.19,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000
12590,63638a6806d67773f3adba8534553fff,4,voucher,1,10.56,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000
21563,63638a6806d67773f3adba8534553fff,6,voucher,1,5.28,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000
74761,63638a6806d67773f3adba8534553fff,1,voucher,1,15.10,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000
79207,63638a6806d67773f3adba8534553fff,7,voucher,1,0.74,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000
88948,63638a6806d67773f3adba8534553fff,2,voucher,1,11.99,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000
96317,63638a6806d67773f3adba8534553fff,3,voucher,1,9.02,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56.000000


In [8]:
mock_payment_log[mock_payment_log['order_id']=='63638a6806d67773f3adba8534553fff']

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,timestamp
74761,63638a6806d67773f3adba8534553fff,1,voucher,1,15.10,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:19:56
88948,63638a6806d67773f3adba8534553fff,2,voucher,1,11.99,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:20:19
96317,63638a6806d67773f3adba8534553fff,3,voucher,1,9.02,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:20:42
12590,63638a6806d67773f3adba8534553fff,4,voucher,1,10.56,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:21:14
7876,63638a6806d67773f3adba8534553fff,5,voucher,1,34.19,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:21:49
21563,63638a6806d67773f3adba8534553fff,6,voucher,1,5.28,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:28
79207,63638a6806d67773f3adba8534553fff,7,voucher,1,0.74,df2988ba3ed226b10521a0e4da849b61,2016-10-04 13:22:56


In [9]:
col_order = [
    'timestamp',
    'order_id',
    'customer_id',
    'payment_sequential',
    'payment_installments',
    'payment_type',
    'payment_value',
]
mock_payment_log = mock_payment_log[col_order]

In [10]:
mock_payment_log.sort_values('timestamp').to_csv(f"{REDEFINED_DIR}/payment.tsv", index=False, sep='\t')